In [18]:
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
plt.rcParams["figure.figsize"] = [20, 15]
input_length=20#koliko dni do danes dobimo podatke?
output_distance=14#za koliko dni vnaprej moramo napovedati stanje?
sample_length=input_length+output_distance

Poberemo podatke o številu okužb po dnevih

In [3]:
path="./csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df=pd.read_csv(path)
width=len(df.columns)-4 #Širina časovnega zaporedja
height=len(df)
df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,24,40,40,74,84,94,110,110,120,170
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,76,89,104,123,146,174,186,197,212,223
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,139,201,230,264,302,367,409,454,511,584
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,88,113,133,164,188,224,267,308,334,370
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,3,3,3,4,4,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Anguilla,United Kingdom,18.2206,-63.0686,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
250,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2
251,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,4,5
252,NaN,MS Zaandam,0.0000,0.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2


Iz teh podatkov naredimo seznam časovnih vrst. 

In [4]:
series=[]
for row in range(height):
    data=[]
    for col in range(4,len(df.columns)):
        data.append(df[df.columns[col]][row])
    series.append(data)
#print(str(series).replace("[","{").replace("]","}"))

Vzamemo neko funkcijo za napako ki je odporna na b=0, ampak vseeno rajši aproksimira relativno napako, ker nas ta precej bolj zanima

In [5]:
def err(x,a):
    return (x-a)**2

Napišemo si funkcijo ki oceni nek (model?) algoritem.

In [6]:
class Data_segment:
    def __init__(self,input_list):
        assert len(input_list)==sample_length
        self.input=input_list[0:input_length]
        self.output=input_list[-1]
        self.raw=input_list[:]

In [7]:
def input_generator():
    inputs=[]
    for row in range(len(series)):
        for c in range(len(series[row])-sample_length):
            if series[row][c]>5:
                list=series[row][c:c+sample_length]
                inputs.append(Data_segment(list))
    return inputs

In [8]:
def evaluate(prediction):
    err_sum=0#vsota relativnih napak
    tries=len(inputs)
    for input in inputs:
        response=prediction(input.input)
        correct=input.output
        error=err(response,correct)
        #print(str(input.input)+" "+str(response)+" "+str(input.ans)+" "+str(error))
        err_sum+=error
    return (err_sum/tries)/1000000

In [9]:
inputs=input_generator()
print(len(inputs))
print(inputs[0].raw)
print(inputs[0].input)
print(inputs[0].output)

1388
[23, 33, 33, 36, 41, 47, 49, 49, 52, 55, 60, 85, 85, 95, 110, 195, 195, 195, 210, 214, 214, 228, 256, 278, 285, 305, 334, 377, 392, 419, 458, 466, 476, 499]
[23, 33, 33, 36, 41, 47, 49, 49, 52, 55, 60, 85, 85, 95, 110, 195, 195, 195, 210, 214]
499


Prvi poskus: število okužb jutri bo 0. To bi moralo dati slab rezultat

In [10]:
def return_healthy_for_sure(ls):
    return 0
print(evaluate(return_healthy_for_sure))

215.81929734870317


Verjetno je precej boljša ideja da vrnemo število okuženih včeraj. Vsaj spodnja meja se ujema...

In [11]:
def return_yesterday(ls):
    return ls[-1]
print(evaluate(return_yesterday))

85.4909355482709


Kot kaže je uporaba malo logike precej boljša. Kdo bi si mislil. Sedaj naredimo interpolacijo iz enega dneva nazaj v "output_distance" dni naprej

In [12]:
def small_interpolation(ls):
    return ls[-1]+output_distance*(ls[-1]-ls[-2])*0.5
print(evaluate(small_interpolation))

73.46838338112391


Poskusimo s pomočjo razmerij

In [13]:
def ratio_interpolation(ls):
    return ls[-1]*(ls[-1]/ls[-2])**(output_distance*0.2)
print(evaluate(ratio_interpolation))

76.13723813668055


Glede na uteži ki smo jih morali dodati da je bil rezultat vsaj približno dober, smo verjetno bistveno divergirali od željenih vrednosti zaradi interpolacije v toliko dni naprej. Poskusimo uporabiti linearno interpolacijo z utežmi

In [14]:
def generate_fit_from_weights_linear(w):
    def linear_weight_interpolation(ls):
        weightedTerms=[w[i]*ls[-1-i] for i in range(len(w))]
        return sum(weightedTerms)
    
    return linear_weight_interpolation

def generate_good_linear_interpolation():
    weights=[0 for i in range(input_length)]#Recimo [0,0,0,0] da prvo funkcijo ki vedno vrne 0, [1,0,0,0] pa da funckijo ki gleda včeraj
    #Iz Jave ?!
    weights=[0.036352221539204546, 0.018249770545115396, 0.47028751427639576, -0.7045054934546577, 0.693455320661259, 0.15158809158411937, 0.16585301704907973, 0.5350970548742747, -1.1301572767617092, 0.6214886149019411, -0.9753954693811485, -0.2901006633541625, 0.28062697194207203, -0.36508154109605706, -0.3318423274094825, 0.24790415394548634, -2.457606529097592, 0.6247383467066236, -2.2728723982148304, 5.7272730939654215][::-1]
    best=evaluate(generate_fit_from_weights_linear(weights))
    print(best)
    last=0
    w=0.5#velikost "povprečne" spremembe
    for i in range(0):
        if(i%100==0):
            print(i)
        p=random.randrange(len(weights))
        change=w*(random.random()-0.5)
        weights[p]+=change
        curr=evaluate(generate_fit_from_weights_linear(weights))
        #print(str(weights)+" "+str(curr))
        if(curr<best):
            best=curr
            if i-last>100:
                last=i
                print(curr)
                print(weights)
        else:
            weights[p]-=change
    print(best)
    return generate_fit_from_weights_linear(weights)
    
good_linear_interpolation=generate_good_linear_interpolation()

68.13037819968498
68.13037819968498


V Javi poskusimo s prileganjem logaritma z kvadratno napako.

In [134]:
w=[0.18028648693431407, -0.008862714613898221, 0.2462799808746143, -0.3624591373940107, -0.2179773166594492, 0.21212835842103114, 0.07060767273438316, -0.14468791339277434, 0.07006490668244514, 0.12963286511599362, -0.09505077538800331, -0.29720509491789154, -0.32575441390618537, -0.23206193348386095, -0.024203801353715186, -0.10121336215746463, -0.7954172840327228, -0.8192345099724636, -0.41803247795289555, 3.95225398846222][::-1]
def log_weight_interpolation(ls):
    if ls[0]<=0:
        return 0
    weightedTerms=[w[i]*math.log(ls[-1-i]) for i in range(len(w))]
    return math.exp(sum(weightedTerms))

print(evaluate(log_weight_interpolation))

124.0402719200734


In [ ]:
while(True):
    row=series[random.randrange(len(series))]
    if(row[-sample_length-5]>20):
        break
        
print(row)
x=np.arange(0, len(row), 1);
y=[]
lin_p=[]
exp_p=[]
for i in range(len(row)):
    y.append(row[i])
    
for i in range(sample_length):
    lin_p.append(row[i])
for i in range(sample_length,len(row)):
    lin_p.append(good_linear_interpolation(y[i-input_length-output_distance:i-output_distance]))
    
for i in range(sample_length):
    exp_p.append(row[i])
for i in range(sample_length,len(row)):
    #print(y[i-input_length-output_distance:i-output_distance])
    #print(log_weight_interpolation(y[i-input_length-output_distance:i-output_distance]))
    exp_p.append(log_weight_interpolation(y[i-input_length-output_distance:i-output_distance]))

errors_lin=[err(lin_p[i],y[i]) for i in range(len(row))]
errors_exp=[err(exp_p[i],y[i]) for i in range(len(row))]
plt.subplot(2, 1, 1)
plt.plot(x, y ,'go', label="Ground")
plt.plot(x, lin_p, label="Linear fit")
plt.plot(x, exp_p, label="Exponential fit")
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(x, errors_lin, label="Linear error")
plt.plot(x, errors_exp, label="Exponential error")
plt.legend()

[2, 7, 18, 18, 36, 72, 109, 109, 162, 240, 286, 333, 391, 476, 548, 600, 661, 698, 740, 771, 804, 844, 872, 900, 913, 925, 930, 933, 934, 934, 934, 934, 934, 934, 934, 934, 934, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 935, 936, 936, 936, 936, 936, 936, 936, 937, 937]
